In [13]:
import datetime
import Model
import numpy as np
import pandas as pd
import math
from multiprocessing import Pool
import os
import warnings
import time
import matplotlib.pyplot as plt
from importlib import reload

reload(Model)
warnings.simplefilter('ignore', FutureWarning)

# positive numbers, start should be lower than end
# a rebalance index represents the month before the rebalancing takes place
# so returns are calculated starting at rebalanceIndex + 1
def rebalanceIndexes(startIndex, endIndex):
    indexes = list(range(startIndex, endIndex, 3))
    indexes.reverse()
    return (indexes)


# get ln returns for an equally balanced portfolio of stocks
def getReturns(portfolio, index, length):
	returns = 0
	print(portfolio)
	print(-1 * index + 1)
	print(-1 * index + length)
	nancount = 0
	for stock in portfolio:
		indReturn = Model.rateOfReturn(Model.retrieveData(stock, 'Last Price', -1 * index + 1, -1 * index + length, []))
		print(stock + ": " + str(indReturn))
		if (not math.isnan(indReturn)):
			returns += indReturn
		else:
			nancount += 1
	if len(portfolio) - nancount == 0:
		total = 0
	else:
		total = returns/(len(portfolio)-nancount)
	print("Start Date: " + str(Model.convertIndexToDate(-1 * index + 1)))
	print("Total Return: " +str(total))
	return (total)


# make a portfolio with predicted probabilities higher than a hardcoded threshold
def makePortfolio(treeTuple):
	featureList = ['EPS Growth', 'Volatility 180 D', 'Trailing EPS', 'Price to Cash Flow', 'EPS', 'Volume', 'Return on Assets', 'Price to Book', 'Dividend Yield', 'Total Debt to Total Equity', 'Return on Invested Capital', 'Return on Common Equity']
	addedStocks, probabilities = Model.predict_probabilities(treeTuple[1], startIndex = -1 * treeTuple[0] - 11, endIndex = -1 * treeTuple[0], features = featureList, sector = "Financials")
	probabilityThreshold = 0.8
	stockTuples = zip(addedStocks, probabilities)
	stockTuples = list(filter(lambda x: x[1][1] > probabilityThreshold, stockTuples))
	if len(stockTuples) == 0:
		print("No portfolio, probabilities lower than threshold of " + str(probabilityThreshold))
		return 0
	stocks, probabilities = zip(*stockTuples)
	return(getReturns(stocks, treeTuple[0], 3))

In [14]:
def poolBuild(trainIndexes):
    t = time.time()
    featureList = ['EPS Growth', 'Volatility 180 D', 'Trailing EPS', 'Price to Cash Flow', 'EPS', 'Volume', 'Return on Assets', 'Price to Book', 'Dividend Yield', 'Total Debt to Total Equity', 'Return on Invested Capital', 'Return on Common Equity']
    forest = Model.buildWithIndexesTripleClass(modelType = Model.randomForestClassifier, indexes = trainIndexes, target= 'Rate of Return', features = featureList, \
                                               featureLength = 12, targetLength = 3, sector = "Financials", percentileTarget = 90, percentileAvoid = 10)
    print("Build time: " + str(time.time()-t) + " seconds.")
    return forest

In [15]:
indexes = []
for i in rebalanceIndexes(4,28):
    maxLength = 200
    targetLength = 3
    featureLength = 12
    indexes.append(np.arange(-1 * (targetLength + featureLength) - i + maxLength * -1, -1 * (targetLength + featureLength) - i, targetLength))
print(len(indexes))
print(indexes)

8
[array([-240, -237, -234, -231, -228, -225, -222, -219, -216, -213, -210,
       -207, -204, -201, -198, -195, -192, -189, -186, -183, -180, -177,
       -174, -171, -168, -165, -162, -159, -156, -153, -150, -147, -144,
       -141, -138, -135, -132, -129, -126, -123, -120, -117, -114, -111,
       -108, -105, -102,  -99,  -96,  -93,  -90,  -87,  -84,  -81,  -78,
        -75,  -72,  -69,  -66,  -63,  -60,  -57,  -54,  -51,  -48,  -45,
        -42]), array([-237, -234, -231, -228, -225, -222, -219, -216, -213, -210, -207,
       -204, -201, -198, -195, -192, -189, -186, -183, -180, -177, -174,
       -171, -168, -165, -162, -159, -156, -153, -150, -147, -144, -141,
       -138, -135, -132, -129, -126, -123, -120, -117, -114, -111, -108,
       -105, -102,  -99,  -96,  -93,  -90,  -87,  -84,  -81,  -78,  -75,
        -72,  -69,  -66,  -63,  -60,  -57,  -54,  -51,  -48,  -45,  -42,
        -39]), array([-234, -231, -228, -225, -222, -219, -216, -213, -210, -207, -204,
       -201, -198,

In [16]:
pool = Pool(os.cpu_count())
print("# of multiprocess cpus: " + str(os.cpu_count()))
forestList = pool.map(poolBuild, indexes)
print(forestList)

# of multiprocess cpus: 8
Finished data retrieval, starting model training.
Finished data retrieval, starting model training.
Finished data retrieval, starting model training.
Finished data retrieval, starting model training.
Finished data retrieval, starting model training.
Finished data retrieval, starting model training.
Finished data retrieval, starting model training.
Finished data retrieval, starting model training.
Finished fitting.
Build time: 1403.2612690925598 seconds.
Finished fitting.
Build time: 1404.7288329601288 seconds.
Finished fitting.
Finished fitting.
Build time: 1407.1074931621552 seconds.
Build time: 1407.1428768634796 seconds.
Finished fitting.
Build time: 1409.8218631744385 seconds.
Finished fitting.
Build time: 1411.3375217914581 seconds.
Finished fitting.
Finished fitting.
Build time: 1416.200001001358 seconds.
Build time: 1416.218001127243 seconds.
[RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, m

In [11]:
pool = Pool(os.cpu_count())
print("# of multiprocess cpus: " + str(os.cpu_count()))
returnsList = pool.map(makePortfolio, zip(rebalanceIndexes(4,28), forestList))
print(returnsList)

# of multiprocess cpus: 8
('AGM', 'AMBC', 'DFS', 'EIG', 'ENVA', 'GWB', 'HASI', 'IVR', 'LTXB', 'NAVI', 'PB', 'SAMG', 'SFBS', 'SMMF', 'TCBI', 'TSC')
-24
-22
AGM: 0.2260361280302603
AMBC: 0.06097126229595862
DFS: 0.19243113259889633
EIG: 0.06755091017005022
ENVA: 0.42148375726380194
GWB: 0.24746940058671374
HASI: 0.09680614356116779
IVR: 0.1276565212131997
LTXB: 0.32879698181477357
NAVI: 0.23288358972295908
PB: 0.26587620391307576
SAMG: 0.17742558618174753
SFBS: 0.29883713302947656
SMMF: 0.34799446607147333
TCBI: 0.34872508528800816
TSC: 0.10652553201660764
Start Date: 2016-02-29
Total Return: 0.22171686460988566
('EARN', 'GNBC', 'LTXB', 'MRLN', 'TREE', 'UBFO', 'WSBF')
-21
-19
EARN: 0.08482782978591841
GNBC: 0.1628798751793692
LTXB: 0.057736090769568094
MRLN: 0.19151400497811855
TREE: 0.19307434857021644
UBFO: 0.11864555025007317
WSBF: 0.07961762796469118
Start Date: 2016-05-31
Total Return: 0.12689933249970786
('AMTD', 'CARO', 'ESNT', 'ETFC', 'FBMS', 'FCFP', 'FFNW', 'FGBI', 'GHL', 'HBMD'

In [12]:
indexes = []
for i in rebalanceIndexes(28,52):
    maxLength = 300
    targetLength = 3
    featureLength = 12
    indexes.append(np.arange(-1 * (targetLength + featureLength) - i + maxLength * -1, -1 * (targetLength + featureLength) - i, targetLength))
print(len(indexes))
print(indexes)

8
[array([-364, -361, -358, -355, -352, -349, -346, -343, -340, -337, -334,
       -331, -328, -325, -322, -319, -316, -313, -310, -307, -304, -301,
       -298, -295, -292, -289, -286, -283, -280, -277, -274, -271, -268,
       -265, -262, -259, -256, -253, -250, -247, -244, -241, -238, -235,
       -232, -229, -226, -223, -220, -217, -214, -211, -208, -205, -202,
       -199, -196, -193, -190, -187, -184, -181, -178, -175, -172, -169,
       -166, -163, -160, -157, -154, -151, -148, -145, -142, -139, -136,
       -133, -130, -127, -124, -121, -118, -115, -112, -109, -106, -103,
       -100,  -97,  -94,  -91,  -88,  -85,  -82,  -79,  -76,  -73,  -70,
        -67]), array([-361, -358, -355, -352, -349, -346, -343, -340, -337, -334, -331,
       -328, -325, -322, -319, -316, -313, -310, -307, -304, -301, -298,
       -295, -292, -289, -286, -283, -280, -277, -274, -271, -268, -265,
       -262, -259, -256, -253, -250, -247, -244, -241, -238, -235, -232,
       -229, -226, -223, -220, -2

In [17]:
pool = Pool(os.cpu_count())
print("# of multiprocess cpus: " + str(os.cpu_count()))
forestListTemp = pool.map(poolBuild, indexes)
print(forestListTemp)
forestListTemp.extend(forestList)
forestList = forestListTemp

# of multiprocess cpus: 8
Finished data retrieval, starting model training.
Finished data retrieval, starting model training.
Finished data retrieval, starting model training.
Finished data retrieval, starting model training.
Finished data retrieval, starting model training.
Finished data retrieval, starting model training.
Finished data retrieval, starting model training.
Finished data retrieval, starting model training.
Finished fitting.
Build time: 6852.680891036987 seconds.
Finished fitting.
Build time: 6861.1635110378265 seconds.
Finished fitting.
Build time: 6865.391268253326 seconds.
Finished fitting.
Finished fitting.
Build time: 6871.3969938755035 seconds.
Build time: 6871.792741060257 seconds.
Finished fitting.
Finished fitting.
Build time: 6874.099924087524 seconds.
Build time: 6874.166500091553 seconds.
Finished fitting.
Build time: 6882.673429965973 seconds.
[RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_f

In [18]:
pool = Pool(os.cpu_count())
print("# of multiprocess cpus: " + str(os.cpu_count()))
returnsListTemp = pool.map(makePortfolio, zip(rebalanceIndexes(28,52), forestList[0:8]))
print(returnsListTemp)
returnsListTemp.extend(returnsList)
returnsList = returnsListTemp

# of multiprocess cpus: 8
('AFSI', 'ENFC', 'FNHC', 'GNW', 'HLI', 'KRNY', 'NATL', 'SNC', 'TBK', 'TRUP', 'TRV', 'UIHC', 'UNTY', 'UVE')
-48
-46
AFSI: 0.02275500299366362
ENFC: 0.16090124243698867
FNHC: 0.20175780757035877
GNW: 0.13858616328614692
HLI: 0.2444978316730757
KRNY: 0.14754670493654576
NATL: 0.14388702538163178
SNC: 0.04889241888602003
TBK: 0.13861012370353842
TRUP: 0.25232955373606103
TRV: 0.07732321755533622
UIHC: 0.15502537920513548
UNTY: 0.08924618099321946
UVE: 0.0908090469235332
Start Date: 2014-02-28
Total Return: 0.13658340709151823
('BHB', 'BLX', 'COBZ', 'FDS', 'GS', 'GSBC', 'HIFS', 'HLI', 'KINS', 'MCO', 'PJC', 'SEIC', 'SNC', 'TRUP', 'UCFC')
-45
-43
BHB: 0.07213949667229791
BLX: 0.10464971710187143
COBZ: 0.08765405053360098
FDS: 0.11443809928046988
GS: 0.07855425733767785
GSBC: 0.06253070278998862
HIFS: 0.12618893732777803
HLI: 0.10230117827447938
KINS: 0.10617187472870526
MCO: 0.01692401585582104
PJC: 0.15865045271985467
SEIC: 0.08412230024429279
SNC: 0.172598860815432

In [19]:
indexes = []
for i in rebalanceIndexes(52,76):
    maxLength = 200
    targetLength = 3
    featureLength = 12
    indexes.append(np.arange(-1 * (targetLength + featureLength) - i + maxLength * -1, -1 * (targetLength + featureLength) - i, targetLength))
print(len(indexes))
print(indexes)

8
[array([-288, -285, -282, -279, -276, -273, -270, -267, -264, -261, -258,
       -255, -252, -249, -246, -243, -240, -237, -234, -231, -228, -225,
       -222, -219, -216, -213, -210, -207, -204, -201, -198, -195, -192,
       -189, -186, -183, -180, -177, -174, -171, -168, -165, -162, -159,
       -156, -153, -150, -147, -144, -141, -138, -135, -132, -129, -126,
       -123, -120, -117, -114, -111, -108, -105, -102,  -99,  -96,  -93,
        -90]), array([-285, -282, -279, -276, -273, -270, -267, -264, -261, -258, -255,
       -252, -249, -246, -243, -240, -237, -234, -231, -228, -225, -222,
       -219, -216, -213, -210, -207, -204, -201, -198, -195, -192, -189,
       -186, -183, -180, -177, -174, -171, -168, -165, -162, -159, -156,
       -153, -150, -147, -144, -141, -138, -135, -132, -129, -126, -123,
       -120, -117, -114, -111, -108, -105, -102,  -99,  -96,  -93,  -90,
        -87]), array([-282, -279, -276, -273, -270, -267, -264, -261, -258, -255, -252,
       -249, -246,

In [21]:
pool = Pool(os.cpu_count())
print("# of multiprocess cpus: " + str(os.cpu_count()))
forestListTemp = pool.map(poolBuild, indexes)
print(forestListTemp)
forestListTemp.extend(forestList)
forestList = forestListTemp

# of multiprocess cpus: 8
Finished data retrieval, starting model training.
Finished data retrieval, starting model training.
Finished data retrieval, starting model training.
Finished data retrieval, starting model training.
Finished data retrieval, starting model training.
Finished data retrieval, starting model training.
Finished data retrieval, starting model training.
Finished data retrieval, starting model training.
Finished fitting.
Build time: 2654.6259648799896 seconds.
Finished fitting.
Build time: 2656.6977519989014 seconds.
Finished fitting.
Build time: 2659.110291957855 seconds.
Finished fitting.
Build time: 2662.9852809906006 seconds.
Finished fitting.
Build time: 2666.059972047806 seconds.
Finished fitting.
Build time: 2670.866875886917 seconds.
Finished fitting.
Build time: 2673.3919973373413 seconds.
Finished fitting.
Build time: 2692.9015080928802 seconds.
[RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, ma

In [ ]:
pool = Pool(os.cpu_count())
print("# of multiprocess cpus: " + str(os.cpu_count()))
returnsListTemp = pool.map(makePortfolio, zip(rebalanceIndexes(52,76), forestList[0:8]))
print(returnsListTemp)
returnsListTemp.extend(returnsList)
returnsList = returnsListTemp

In [ ]:
indexes = []
for i in rebalanceIndexes(76,100):
    maxLength = 300
    targetLength = 3
    featureLength = 12
    indexes.append(np.arange(-1 * (targetLength + featureLength) - i + maxLength * -1, -1 * (targetLength + featureLength) - i, targetLength))
print(len(indexes))
print(indexes)

In [ ]:
pool = Pool(os.cpu_count())
print("# of multiprocess cpus: " + str(os.cpu_count()))
forestListTemp = pool.map(poolBuild, indexes)
print(forestListTemp)
forestListTemp.extend(forestList)
forestList = forestListTemp

In [ ]:
pool = Pool(os.cpu_count())
print("# of multiprocess cpus: " + str(os.cpu_count()))
returnsListTemp = pool.map(makePortfolio, zip(rebalanceIndexes(76,100), forestList[0:8]))
print(returnsListTemp)
returnsListTemp.extend(returnsList)
returnsList = returnsListTemp

In [ ]:
indexes = []
for i in rebalanceIndexes(100,124):
    maxLength = 300
    targetLength = 3
    featureLength = 12
    indexes.append(np.arange(-1 * (targetLength + featureLength) - i + maxLength * -1, -1 * (targetLength + featureLength) - i, targetLength))
print(len(indexes))
print(indexes)

In [ ]:
pool = Pool(os.cpu_count())
print("# of multiprocess cpus: " + str(os.cpu_count()))
forestListTemp = pool.map(poolBuild, indexes)
print(forestListTemp)
forestListTemp.extend(forestList)
forestList = forestListTemp

In [ ]:
pool = Pool(os.cpu_count())
print("# of multiprocess cpus: " + str(os.cpu_count()))
returnsListTemp = pool.map(makePortfolio, zip(rebalanceIndexes(100,124), forestList[0:8]))
print(returnsListTemp)
returnsListTemp.extend(returnsList)
returnsList = returnsListTemp

In [ ]:
indexes = []
for i in rebalanceIndexes(124,148):
    maxLength = 300
    targetLength = 3
    featureLength = 12
    indexes.append(np.arange(-1 * (targetLength + featureLength) - i + maxLength * -1, -1 * (targetLength + featureLength) - i, targetLength))
print(len(indexes))
print(indexes)

In [ ]:
pool = Pool(os.cpu_count())
print("# of multiprocess cpus: " + str(os.cpu_count()))
forestListTemp = pool.map(poolBuild, indexes)
print(forestListTemp)
forestListTemp.extend(forestList)
forestList = forestListTemp

In [ ]:
pool = Pool(os.cpu_count())
print("# of multiprocess cpus: " + str(os.cpu_count()))
returnsListTemp = pool.map(makePortfolio, zip(rebalanceIndexes(124,148), forestList[0:8]))
print(returnsListTemp)
returnsListTemp.extend(returnsList)
returnsList = returnsListTemp

In [ ]:
indexes = []
for i in rebalanceIndexes(148,172):
    maxLength = 300
    targetLength = 3
    featureLength = 12
    indexes.append(np.arange(-1 * (targetLength + featureLength) - i + maxLength * -1, -1 * (targetLength + featureLength) - i, targetLength))
print(len(indexes))
print(indexes)

In [ ]:
pool = Pool(os.cpu_count())
print("# of multiprocess cpus: " + str(os.cpu_count()))
forestListTemp = pool.map(poolBuild, indexes)
print(forestListTemp)
forestListTemp.extend(forestList)
forestList = forestListTemp

In [ ]:
pool = Pool(os.cpu_count())
print("# of multiprocess cpus: " + str(os.cpu_count()))
returnsListTemp = pool.map(makePortfolio, zip(rebalanceIndexes(148,172), forestList[0:8]))
print(returnsListTemp)
returnsListTemp.extend(returnsList)
returnsList = returnsListTemp

In [ ]:
indexes = []
for i in rebalanceIndexes(172,196):
    maxLength = 300
    targetLength = 3
    featureLength = 12
    indexes.append(np.arange(-1 * (targetLength + featureLength) - i + maxLength * -1, -1 * (targetLength + featureLength) - i, targetLength))
print(len(indexes))
print(indexes)

In [ ]:
pool = Pool(os.cpu_count())
print("# of multiprocess cpus: " + str(os.cpu_count()))
forestListTemp = pool.map(poolBuild, indexes)
print(forestListTemp)
forestListTemp.extend(forestList)
forestList = forestListTemp

In [ ]:
pool = Pool(os.cpu_count())
print("# of multiprocess cpus: " + str(os.cpu_count()))
returnsListTemp = pool.map(makePortfolio, zip(rebalanceIndexes(172,196), forestList[0:8]))
print(returnsListTemp)
returnsListTemp.extend(returnsList)
returnsList = returnsListTemp

In [ ]:
print(returnsList)
print("Mean: " + str(np.mean(returnsList)))
print("Std Dev: " + str(np.std(returnsList)))
print("Total Sum: " + str(sum(returnsList)))
plt.hist(returnsList, 8, facecolor='green', alpha=0.75)